# Understanding of Sequence to Sequence Model
Sequence to Sequence is the most researched area in the era of modern NLP. The sequence to Sequence task is one size fit all kind of algorithm which is used in all below given listed tasks.

1. Machine Translation
2. Summarization
3. Question Answering
4. Chat-bot
5. Text Simplification
6. Speech to text
7. text to speech

In this implemetation, we will start off with general Sequence to Sequence architecture, then one by one we will implement GRU encoder-decoder, batching with encoder-decoder and batch processing with attention. This implementation will be discussed by tightly integrating algorithm, equations, and code. The discussion in the upcoming recipe forms the base of modern NLP techniques. To give you a flavor of how to do text processing from scratch, I am not using TorchText into these recipes. 

The sequence to Sequence network contains two parts an encoder and a decoder. The encoder takes the given input converts it into the fixed size hidden representation. This hidden representation will be converted to output by a decoder.  An Encoder takes input sequence and passes through the Embedding Layer.  Embedding Layer converts word into a fixed size vector, this Embedding Layer is trainable and get trained along with encoder and decoder. 


# Dataset

The data for this imlementation is a set of many thousands of  French  to English translation pairs. These pairs are shared from  http://www.manythings.org/anki/fra-eng.zip. Dataset is already present at `data/fra-eng.txt`

|Fra|Eng|
|:---:|:---:|
|Va !|Go.|
|Cours !|Run!|
|Saute.|Jump.|
|Ça suffit !|Stop!|
|Stop !|Stop!|
|Arrête-toi !|Stop!|
|Attends !|Wait!|
|J'ai froid. | I am cold.|


Such 11,000+ pairs for french to English traslation are given in original dataset.

In [ ]:
import random
import re
import time
import unicodedata

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn.functional as F
from torch import nn, optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

# Data Preparation

It better to **reuse**, I am reusing official text preprocessing [code](https://github.com/pytorch/tutorials) written for seq2seq tutorial.

In [ ]:
SOS_token = 0
EOS_token = 1
MAX_LENGTH = 10

To keep track of all this we will use a helper class called Lang which has word → index (word2index) and index → word (index2word) dictionaries, as well as a count of each word word2count to use to later replace rare words.

In [ ]:
class Lang(object):
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

The files are all in Unicode, to simplify we will turn Unicode characters to ASCII, make everything lowercase, and trim most punctuation.

In [ ]:
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn')


# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

To read the data file we will split the file into lines, and then split lines into pairs. The files are all English → Other Language, so if we want to translate from Other Language → English I added the `reverse` flag to reverse the pairs.

In [ ]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('data/fra-eng.txt', encoding='utf-8').read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

Since there are a lot of example sentences and we want to train something quickly, we’ll trim the data set to only relatively short and simple sentences. Here the maximum length is 10 words (that includes ending punctuation) and we’re filtering to sentences that translate to the form “I am” or “He is” etc. (accounting for apostrophes replaced earlier).

In [ ]:
eng_prefixes = ("i am ", "i m ", "he is", "he s ", "she is", "she s",
                "you are", "you re ", "we are", "we re ", "they are",
                "they re ")


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

The full process for preparing the data is:

- Read text file and split into lines, split lines into pairs
- Normalize text, filter by length and content
- Make word lists from sentences in pairs

In [ ]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    pairs = filterPairs(pairs)
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Input Language : ",input_lang.name, ", Number of words : " ,input_lang.n_words)
    print("Target Language : ",output_lang.name, ", Number of words : " ,output_lang.n_words)
    print("A Random Pair : ",random.choice(pairs))
    return input_lang, output_lang, pairs


def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    result = torch.LongTensor(indexes)
    return result


def tensorFromPair(input_lang, output_lang, pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return input_tensor, target_tensor

To train, for each pair we will need an input tensor (indexes of the words in the input sentence) and target tensor (indexes of the words in the target sentence). While creating these vectors we will append the EOS token to both sequences.

In [ ]:
class TextDataset(Dataset):
    def __init__(self, dataload=prepareData, lang=['eng', 'fra']):
        self.input_lang, self.output_lang, self.pairs = dataload(
            lang[0], lang[1], reverse=True)
        self.input_lang_words = self.input_lang.n_words
        self.output_lang_words = self.output_lang.n_words

    def __getitem__(self, index):
        return tensorFromPair(self.input_lang, self.output_lang,
                              self.pairs[index])

    def __len__(self):
        return len(self.pairs)

In [ ]:
lang_dataset = TextDataset()
lang_dataloader = DataLoader(lang_dataset, shuffle=True)

One can access pairs of `French` and `English` by usinf iterator `lang_dataloader`.

In [ ]:
for i, data in enumerate(lang_dataloader):
    in_lang, out_lang = data
    print(in_lang, out_lang)
    break

In [ ]:
input_size = lang_dataset.input_lang_words
hidden_size = 256
output_size = lang_dataset.output_lang_words
MAX_LENGTH = 10

# Some Learning

**Some basic understanding regarding PyTorch componenets used in Encoder or Decoder are  explianed below : **

## Embeddings

To learn language one must convert words to fixed size vectors. This can be done is two ways:

1. Using pretrained word vectors like Word2vec or Glove Vectors
2. Learning Vector from Scratch

Here we will be using learning Vector from Scratch, Pytorch has the **Embedding** function for the same. These embeddings will be trained as when learning takes place. Below given is the example how one can insert PyTorch Embedding layer in the model. 
For Example: We have two words in vocab "hello" and "world" and we want to have 5 dimentional vector for each word then PyTorch Embedding  can be defined in following way: [[2](https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html)]

In [ ]:
word_to_ix = {"hello": 0, "world": 1}
embeds = nn.Embedding(2, 5)  # 2 words in vocab, 5 dimensional embeddings
lookup_tensor = torch.tensor([word_to_ix["hello"]], dtype=torch.long)
hello_embed = embeds(lookup_tensor)
print(hello_embed)

## Unsqueeze & Squeeze

### Unsqueeze
unsqueeze() inserts singleton dim at position given as parameter. Insert a new axis that will appear at the axis position in the expanded array shape.[[3](https://pytorch.org/docs/stable/torch.html#torch.unsqueeze)]



In [ ]:
input = torch.Tensor(2, 4, 3) # input: 2 x 4 x 3
print(input.unsqueeze(0).size())

### Squeeze
Returns a tensor with all the dimensions of input of size 1 removed. For example, if input is of shape: $(A×1×B×C×1×D)$ then the out tensor will be of shape: $(A×B×C×D)$.[[4](https://pytorch.org/docs/stable/torch.html#torch.squeeze)]


In [ ]:
x = torch.zeros(2, 1, 2, 1, 2) # input: 2 x 4 x 3
y = torch.squeeze(x)
y.size()

## Permute
Interchange different axis. [[5](https://discuss.pytorch.org/t/permute-elements-of-a-tensor-along-a-dimension/1105)]

In [ ]:
x = torch.randn(2, 3, 5)
torch.Size([2, 3, 5])
print(x.permute(2, 0, 1).size())

## LogSoftmax
Log Softmax applies logarithm after softmax : 
$log( exp(x_i) / exp(x).sum() )$ [[6](https://discuss.pytorch.org/t/what-is-the-difference-between-log-softmax-and-softmax/11801)]

## TopK
Returns the k largest elements of the given input tensor along a given dimension.
If dim is not given, the last dimension of the input is chosen.
If largest is False then the k smallest elements are returned.A tuple of (values, indices) is returned, where the indices are the indices of the elements in the original input tensor. [[7](https://pytorch.org/docs/stable/torch.html#torch.topk)]

In [ ]:
x = torch.arange(1., 10.) #tensor([ 1.,  2.,  3.,  4.,  5.])
top_value, top_index = torch.topk(x, 3)
print(top_value, top_index)

## GRU
Applies a multi-layer gated recurrent unit (GRU) RNN to an input sequence. [[8](https://pytorch.org/docs/stable/nn.html#torch.nn.GRU)]

#Learning Model
With understanding of above elements, we are good to go with defining Encoder and Decoder Network.

![Encoder Decoder Architecure for Sequence to Sequence Learning](figures/encoder_decoder.png)

Figure 1. Encoder Decoder architecure while training

An Encoder takes **input sequence** passes through the **Embedding Layer**. **Embedding Layer** converts word in to a fixed size vector, this Embedding Layer is trainable and get trained along with encoder and decoder. Here onward each word is in form of Embedded representation and passed to GRU. GRU takes 2 inputs 1) Encoder Hidden states and 2) Embedded word input. 
All words of the input sentence are passed sequentially to GRU so that it takes for a word it takes hidden state of previous state as input and gives one output. For now I will be ignoring encoder outputs shown by :negative_squared_cross_mark: . (These Encoder outputs, we will be using when we implement attention mechanism)  

Encoder and Decoder states are represented by Dotted Blue arrow throughout. Embedding representaion are shown in pink. 

After Encoder phase has finished, we her final encoder hidden state as output. This **encoder hidden state is of value** and it is estimated that as it has seen all the word of the sequence it will be having information regarding entire sequence.

This Encoder hidden state is passed to Decoder as first hidden state. 

**Concept Of Teacher Forcing :** 

Teacher Forcing is kind of **Hint** in training recurrent neural networks that uses model output from a prior time step as an input to next time step. 
For Example, 
  
1. Input sentence for encoder is "Mon nom est Sunil" <- (French)
2. Actual Target Sentence is "My name is Sunil" <- (English)
3. In Decoding step at first time step **<Go\>** token is given as input to GRU along with  context vector.**<GO\> **or **<SOS\>** mark begining of decoding and looking at context vector it will generate first output word in English.Then GRU will generate (probably)**"My"**.
4. Now in second time step "**My"** will be given as input along with previous time step hidden step. This will give output as **"name"**.
5. Generation wil continues as long as **<EOS\>** token is encountered.  

At training time we know the output sequence for the input sequecne so we provide words of actual output for teacher forcing.
At test time we dont know the output sequence for the input sequecne so output token generated at time step $t$ is given as input to $t+1$ step (As shown in Figure 2)

    





### Encoder

In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, n_layers=1):
        super(EncoderRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        # embed size = hidden size for simplicity
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        input = input.unsqueeze(1)
        embedded = self.embedding(input)  # batch, hidden
        output = embedded.permute(1, 0, 2)
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        # batch size is 1  and number of layers is 1 so the hidden state size is (1,1,hidden size)
        result = Variable(torch.zeros(1, 1, self.hidden_size))
        return result

#### Testing Encoder

In [ ]:
encoder_test = EncoderRNN(input_size, hidden_size)
test_encoder_input = torch.tensor([15])
encoder_hidden = encoder_test.initHidden()
encoder_output, encoder_hidden = encoder_test(test_encoder_input,encoder_hidden)
print(" Encoder output shape : ", encoder_output.shape)

### Decoder
Decoder has the similar structure as encoder. It has Embedding layer that convert input to dense vectors. These dense vectors are then passed to the GRU layer. The first time step of the Decoder receive  context vector as the hidden state vector. A linear transformation is applied that converts the output from GRU.  This linear transformation converts the GRU output in to size equal to output/target language  vocabulary size. A soft-max is applied to this layer and the values are converted to probabilities. Highest probability for any word indicates that  a particular word is being generated as output  by decoder. 

In [ ]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, n_layers=1):
        super(DecoderRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        #Embeding size  = hidden size for simplicity
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax()

    def forward(self, input, hidden):
        output = self.embedding(input)  # batch, 1, hidden
        output = output.permute(1, 0, 2)  # 1, batch, hidden
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        # batch size is 1  and number of layers is 1 so the hidden state size is (1,1,hidden size)
        result = Variable(torch.zeros(1, 1, self.hidden_size))
        return result


#### Testing Decoder

In [ ]:
decoder_test = DecoderRNN(hidden_size,output_size)
decoder_hidden = decoder_test.initHidden()
test_decoder_input = torch.tensor([[13]])
decoder_output, test_hidden = decoder_test(test_decoder_input,encoder_hidden)
print("Decoder output shape : ",decoder_output.shape)

### Plotting Function

In [ ]:
def showPlot(points):
    plt.figure()
    plt.xlabel("Iterations")
    plt.ylabel("Loss")
    x = np.arange(len(points))
    plt.plot(x, points)
    plt.show()

# Actual Training

Training has following component involved in sequential manner


1.    Collecting all trainable components for encoder and decoder. Providing all these components to the optimizer.
2. Defining loss function.
3. Using out data loader to get source and target sentences. 
4. Initialising encoder hidden state.
5. Running through all the tokens in the source sentence and generating final encoder hidden state.  
6. Passing final encoder hidden state to the decoder along with start of sequence token as input. 
7. Run the decoder and allow it to generate token by using teacher forcing.
8. When <EOS> token is encountered stop break the decoding loop.
9. Calculate loss by comparing generated output with the actual output. 
10. back-propagate the loss and change the weights.
11. Optionally track loss to visualise if the network is converging or not. 
12. Repeat step 3 to 10 till your all sentence pair get processed in each epoch.
13. Run this training for n epochs and Evaluate.

In [ ]:
encoder = EncoderRNN(input_size, hidden_size)
decoder = DecoderRNN(hidden_size, output_size, n_layers=2)

In [ ]:
def train(encoder, decoder, total_epoch):
    # Collecting all trianable parameters
    param = list(encoder.parameters()) + list(decoder.parameters())
    # Defining Optimizer
    optimizer = optim.Adam(param, lr=1e-3)
    # Defining Loss Function
    criterion = nn.NLLLoss()

    plot_losses = []

    # Learning for defined total_epoch
    for epoch in range(total_epoch):
        since = time.time()
        running_loss = 0
        print_loss_total = 0
        total_loss = 0

        # getting french and english pairs by iterating over data iterator
        for i, data in enumerate(lang_dataloader):
            in_lang, out_lang = data

            in_lang = Variable(in_lang)
            out_lang = Variable(out_lang)

            # Passing data to Encoder 
            encoder_outputs = Variable(torch.zeros(MAX_LENGTH, encoder.hidden_size))
            encoder_hidden = encoder.initHidden()

            for encoder_input in range(in_lang.size(1)):
                encoder_output, encoder_hidden = encoder(torch.tensor([in_lang[0][encoder_input]]), encoder_hidden)

            # Passing data to Decoder 
            decoder_input = Variable(torch.LongTensor([[SOS_token]]))
            # using Encoder's last state as Decoder's last state
            decoder_hidden = encoder_hidden
            loss = 0
            for decoder_input_no in range(out_lang.size(1)):
                decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
                loss += criterion(decoder_output, torch.tensor([out_lang[0][decoder_input_no]]))
                topv, topi = decoder_output.data.topk(1)
                ni = int(topi[0][0])
                decoder_input = Variable(torch.LongTensor([[ni]]))
                if ni == EOS_token:
                    break

            # backward operations
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.data[0]
            print_loss_total += loss.data[0]
            total_loss += loss.data[0]
            if (i + 1) % 500 == 0:
                print('Iterarion {}/{}, Loss:{:.6f}'.format(i + 1, len(lang_dataloader), running_loss / 5000))
                running_loss = 0
            if (i + 1) % 100 == 0:
                plot_loss = print_loss_total / 100
                plot_losses.append(plot_loss)
                print_loss_total = 0
        during = time.time() - since
        print('Epoch {}/{} , Loss:{:.6f}, Time:{:.0f}s'.format(epoch + 1, total_epoch, total_loss / len(lang_dataset),during))
    
    #plotting loss
    showPlot(plot_losses)

In [ ]:
# running training
total_epoch = 1
train(encoder, decoder, total_epoch)

# Evaluation

Evaluation module follows the same flow as training, it takes pre-trained encoder and decoder and generate translated text. The main thing here is teacher forcing is not used while evaluation. The decoder generate a word at each time step and the same word is feed as input to next time step. At test time we dont know the output sequence for the input sequence so output token generated at time step t is given as input to t+1 step.

![alt text](figures/Encoder_decoder_testing.png)

Figure 2. Encoder Decoder architecure while testing.

At test time we dont know the output sequence for the input sequecne so output token generated at time step $t$ is given as input to $t+1$ step

In [ ]:
def evaluate(encoder, decoder, in_lang, max_length=MAX_LENGTH):
    input_variable = Variable(in_lang)
    input_variable = input_variable.unsqueeze(0)
    input_length = input_variable.size(1)
    # Encoder Phase
    encoder_hidden = encoder.initHidden()
    encoder_outputs = Variable(torch.zeros(max_length, encoder.hidden_size))
    encoder_outputs = encoder_outputs
    for encoder_input in range(input_length):
        encoder_output, encoder_hidden = encoder(torch.tensor([input_variable[0][encoder_input]]), encoder_hidden)
    
    #Decoder Phase 
    decoder_input = Variable(torch.LongTensor([[SOS_token]]))
    decoder_input = decoder_input
    decoder_hidden = encoder_hidden
    decoded_words = []
    decoder_attentions = torch.zeros(max_length, max_length)

    for decoder_input_no in range(max_length):
        decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
        topv, topi = decoder_output.data.topk(1)
        ni = int(topi[0][0])
        # if decoder gives End Of Sequence token then break the sequence generation else continue.
        if ni == EOS_token:
            break
        else:
            decoded_words.append(lang_dataset.output_lang.index2word[ni])

        decoder_input = Variable(torch.LongTensor([[ni]]))
        decoder_input = decoder_input
    return decoded_words


def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair_idx = random.choice(list(range(len(lang_dataset))))
        pair = lang_dataset.pairs[pair_idx]
        in_lang, out_lang = lang_dataset[pair_idx]
        output_words = evaluate(encoder, decoder, in_lang)
        output_sentence = ' '.join(output_words)
        print('Input : ', pair[0], ' | Desired Output : ', pair[1], ' | Generated Output : ', output_sentence)


In [ ]:
## Randomly evaluating output

In [ ]:
evaluateRandomly(encoder, decoder)